In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='1'
import torch.nn as nn
from torchinfo import summary
from torch.nn import functional as F
import os


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5,dilation=2)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5,dilation=2)
        self.conv3 = nn.Conv2d(20, 30, kernel_size=5,dilation=2)
        self.conv4 = nn.Conv2d(30, 40, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(7200, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.mish(F.max_pool2d(self.conv1(x), 2))
        x = F.mish(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = F.mish(F.max_pool2d(self.conv2_drop(self.conv3(x)), 2))
        x = F.mish(F.max_pool2d(self.conv2_drop(self.conv4(x)), 2))
        
        
        x = x.view(-1, 7200)
        x = F.mish(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [2]:
model = Net()
summary(model)

Layer (type:depth-idx)                   Param #
Net                                      --
├─Conv2d: 1-1                            760
├─Conv2d: 1-2                            5,020
├─Conv2d: 1-3                            15,030
├─Conv2d: 1-4                            30,040
├─Dropout2d: 1-5                         --
├─Linear: 1-6                            360,050
├─Linear: 1-7                            510
Total params: 411,410
Trainable params: 411,410
Non-trainable params: 0

In [3]:
print(model)

Net(
  (conv1): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1), dilation=(2, 2))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1), dilation=(2, 2))
  (conv3): Conv2d(20, 30, kernel_size=(5, 5), stride=(1, 1), dilation=(2, 2))
  (conv4): Conv2d(30, 40, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=7200, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


In [4]:
summary(model, input_size=(5,3,198,198))

Layer (type:depth-idx)                   Output Shape              Param #
Net                                      --                        --
├─Conv2d: 1-1                            [5, 10, 190, 190]         760
├─Conv2d: 1-2                            [5, 20, 87, 87]           5,020
├─Dropout2d: 1-3                         [5, 20, 87, 87]           --
├─Conv2d: 1-4                            [5, 30, 35, 35]           15,030
├─Dropout2d: 1-5                         [5, 30, 35, 35]           --
├─Conv2d: 1-6                            [5, 40, 13, 13]           30,040
├─Dropout2d: 1-7                         [5, 40, 13, 13]           --
├─Linear: 1-8                            [1, 50]                   360,050
├─Linear: 1-9                            [1, 10]                   510
Total params: 411,410
Trainable params: 411,410
Non-trainable params: 0
Total mult-adds (M): 444.97
Input size (MB): 2.35
Forward/backward pass size (MB): 22.24
Params size (MB): 1.65
Estimated Total Size (M